In [16]:
import pandas as pd

In [17]:
# Load CSV and drop the 'id' column
train_csv = pd.read_csv("train_images.csv")
test_csv = pd.read_csv("test_images_path.csv")
test_csv = test_csv.drop(columns=['id'])

In [18]:
print(((train_csv['label']).value_counts()).describe())
len(train_csv['label'].unique())
#train_csv.count()

count    200.00000
mean      19.63000
std        8.39377
min        5.00000
25%       12.75000
50%       20.00000
75%       27.00000
max       35.00000
Name: count, dtype: float64


200

In [19]:
print(((test_csv['label']).value_counts()).describe())
len(test_csv['label'].unique())


count       1.0
mean     4000.0
std         NaN
min      4000.0
25%      4000.0
50%      4000.0
75%      4000.0
max      4000.0
Name: count, dtype: float64


1

In [20]:
for _, row in train_csv.iterrows():
        print('here'+row[0])

C:\Users\martj\AppData\Local\Temp\ipykernel_12424\799708236.py:2: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  print('here'+row[0])


here/train_images/1.jpg
here/train_images/2.jpg
here/train_images/3.jpg
here/train_images/4.jpg
here/train_images/5.jpg
here/train_images/6.jpg
here/train_images/7.jpg
here/train_images/8.jpg
here/train_images/9.jpg
here/train_images/10.jpg
here/train_images/11.jpg
here/train_images/12.jpg
here/train_images/13.jpg
here/train_images/14.jpg
here/train_images/15.jpg
here/train_images/16.jpg
here/train_images/17.jpg
here/train_images/18.jpg
here/train_images/19.jpg
here/train_images/20.jpg
here/train_images/21.jpg
here/train_images/22.jpg
here/train_images/23.jpg
here/train_images/24.jpg
here/train_images/25.jpg
here/train_images/26.jpg
here/train_images/27.jpg
here/train_images/28.jpg
here/train_images/29.jpg
here/train_images/30.jpg
here/train_images/31.jpg
here/train_images/32.jpg
here/train_images/33.jpg
here/train_images/34.jpg
here/train_images/35.jpg
here/train_images/36.jpg
here/train_images/37.jpg
here/train_images/38.jpg
here/train_images/39.jpg
here/train_images/40.jpg
here/trai

DATA PREPROCESSING

    Resize images (e.g., 224x224).
    Normalize pixel values to range [0, 1].
    Apply data augmentation (rotation, flipping, etc.). 

In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
import os
import numpy as np
from sklearn.model_selection import train_test_split


In [22]:

def preprocess_data(csv_data, base_folder='', img_size=(224, 224)):
    """
    Preprocess images based on CSV paths and labels.
    Args:
        csv_data: Pandas DataFrame with 'path' and 'label' columns.
        base_folder: Base folder containing the images (e.g., "train_images").
        img_size: Target size for resizing images.
    Returns:
        images: Numpy array of image data.
        labels: Numpy array of labels.
    """
    images = []
    labels = []
    for _, row in csv_data.iterrows():
        img_path = (base_folder + row[0])  # Add base folder to path
        label = row[1]  # Label

        # Load and preprocess the image
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=img_size)
        img_array = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalize
        images.append(img_array)
        labels.append(label)
    
    return np.array(images), np.array(labels)


# Preprocess training data
train_images, train_labels = preprocess_data(train_csv, base_folder="C:/Users/martj/Desktop/Applied ML/AML-assignment/train_images")

# Preprocess testing data
#test_images, test_labels = preprocess_data(test_csv, base_folder="C:/Users/martj/Desktop/Applied ML/AML-assignment/test_images")


# Convert labels to one-hot encoding
num_classes = len(np.unique(train_labels))

# Adjust labels to 0-based indexing
train_labels = train_labels - 1
print(len(train_labels))
train_labels = to_categorical(train_labels, num_classes=200)
#test_labels = to_categorical(test_labels, num_classes=num_classes)


C:\Users\martj\AppData\Local\Temp\ipykernel_12424\2591305594.py:15: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  img_path = (base_folder + row[0])  # Add base folder to path
C:\Users\martj\AppData\Local\Temp\ipykernel_12424\2591305594.py:16: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  label = row[1]  # Label


3926


In [23]:
# Find unique one-hot encoded vectors
unique_vectors = np.unique(train_labels, axis=0)
print(len(unique_vectors))  # This will give the number of unique vectors


200


In [24]:
# First, split into training + validation and test sets
train_images, test_images, train_labels, test_labels = train_test_split(
    train_images, train_labels, test_size=0.2, random_state=42, stratify=train_labels
)

# Next, split the training data into training and validation sets
train_images, val_images, train_labels, val_labels = train_test_split(
    train_images, train_labels, test_size=0.25, random_state=42, stratify=train_labels
)  # 0.25 x 0.8 = 0.2 of the original data

In [25]:
print("train data size", len(train_labels))
print("test data size", len(test_labels))
print("validation data size", len(val_labels))

train data size 2355
test data size 786
validation data size 785


In [ ]:
# Find unique one-hot encoded vectors
unique_vectors = np.unique(train_labels, axis=0)
print("unique labels in training data", len(unique_vectors))  # This will give the number of unique vectors
# Find unique one-hot encoded vectors
unique_vectors = np.unique(test_labels, axis=0)
print("unique labels in test data", len(unique_vectors))  # This will give the number of unique vectors
# Find unique one-hot encoded vectors
unique_vectors = np.unique(val_labels, axis=0)
print("unique labels in validation data", len(unique_vectors))  # This will give the number of unique vectors

unique labels in training data 200
unique labels in test data 200
unique labels in validation data 200


**Define data generators**

In [ ]:
####Can mess with this too, although I did not see much change when doing so 
train_datagen = ImageDataGenerator(
    #Randomly rotates the image within a range of ±20 degrees
    rotation_range=20,
    #Shifts the image horizontally by a random fraction (up to 20% of the width)
    width_shift_range=0.2,
    #Shifts the image vertically by a random fraction (up to 20% of the height)
    height_shift_range=0.2,
    #Applies a random shearing transformation within 0.2 radians
    shear_range=0.2,
    #Zooms in or out on the image randomly by up to 20%
    zoom_range=0.2,
    # Randomly flips the image horizontally
    horizontal_flip=True,
    #Fills in pixels that are generated due to transformations using the nearest pixel values
    fill_mode='nearest'
)

# Apply data augmentation to training images only
train_generator = train_datagen.flow(
    train_images, train_labels, batch_size=32
)

val_datagen = ImageDataGenerator()  # No augmentation for validation
val_generator = val_datagen.flow(
    val_images, val_labels, batch_size=32
)


In [ ]:
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D

# Load Pre-trained Model
# Maybe look for different pretrained models as well
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Add Custom Layers
### More layers for more complexity ###
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(512, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Define the Model
model = Model(inputs=base_model.input, outputs=predictions)

# Freeze Base Layers for Transfer Learning
# for layer in base_model.layers:
#     layer.trainable = False

for layer in base_model.layers[-50:]:  # Unfreeze the last 50 layers
    layer.trainable = True


# Compile Model
#smaller learning rate when fine-tuning to avoid destroying pre-trained weights
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5), loss='categorical_crossentropy', metrics=['accuracy'])


In [29]:
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=10,
    steps_per_epoch=len(train_images) // 32,
    validation_steps=len(val_images) // 32
)


Epoch 1/10
73/73 [==============================] - 210s 3s/step - loss: 5.5143 - accuracy: 0.0099 - val_loss: 5.4632 - val_accuracy: 0.0104
Epoch 2/10
73/73 [==============================] - 166s 2s/step - loss: 5.3430 - accuracy: 0.0116 - val_loss: 5.3643 - val_accuracy: 0.0143
Epoch 3/10
73/73 [==============================] - 178s 2s/step - loss: 5.2273 - accuracy: 0.0215 - val_loss: 5.2890 - val_accuracy: 0.0182
Epoch 4/10
14/73 [====>.........................] - ETA: 3:35 - loss: 5.1177 - accuracy: 0.0246

KeyboardInterrupt: 

In [15]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_images, test_labels, batch_size=32)
print(f"Test Accuracy: {test_accuracy * 100:.2f}%")


25/25 [==============================] - 11s 430ms/step - loss: 2.3291 - accuracy: 0.4160
Test Accuracy: 41.60%


In [114]:
model.save('bird_species_classifier.h5')
